## Yelp Data Scrapping

> ### Inputs Cell.  

In [1]:
search_subject = "accountants"
search_location = 'San Francisco, CA'# make a check that search makes the 
pages_to_scrape = 5

> ### Dependecies Cell.

In [2]:
url ="https://www.yelp.com/search?find_desc=accountants&find_loc=San+Francisco%2C+CA"
from httpx_html import HTMLSession
import time
from time import sleep
import pandas as pd
import numpy as np
import time
from time import sleep
import pandas as pd
import numpy as np
import json

import datetime
from fake_useragent import UserAgent
import os, glob
from tqdm import tqdm

> ### Main Scraper Cell.

In [3]:
last_df = True
result_list =[]
next = False
site = url
breaker = False
previous_page = url
results_scraped = 0
save_index = 0###
I = 0###
date_time = str( f'{datetime.datetime.now()}')[:f'{datetime.datetime.now()}'.index('.')].replace(':','.')###

In [8]:
start_time = time.time()


ua = UserAgent()
header = {'User-Agent': ua.random}

""" scraping the internet"""      
""" scraping the internet"""      
session = HTMLSession()
r = session.get(url)
response = r.html

results_available = int(response.find('div[class*="text-align--center"] > span')[0].text.split(" ")[-1]) # returns the number of resluts available for a given search
print(f'{results_available} pages available.')

try:
    pages_to_scrape =list(range(pages_to_scrape+1))[I:]
except:
    pages_to_scrape = pages_to_scrape[I:]
    
if len(pages_to_scrape) > results_available:
   pages_to_scrape = pages_to_scrape[:results_available -1]
   
 
for Page in tqdm(pages_to_scrape ,unit= "page", ncols = 160, colour= '#0A9092'):
    
    if next == True:
        ua = UserAgent()
        header = {
                'User-Agent': ua,
                'Origin': 'https://www.yellowpages.com',
                'Referer':f'{previous_page}',
                }
        while True:
            try:
                
                try:
                    breaker = open(f'breaker.txt','r')
                    breaker = True
                    I = save_index
                    break
                except:
                    pass
                
                session = HTMLSession()
                r = session.get(site)
                response = r.html
                break
            except:
                # ua = UserAgent().random
                sleep(5)  
    if breaker == True:
        break
    
    non_sponsored = response.xpath('//h3/span/a')
        
    if len(non_sponsored)>= 10:
        non_sponsored = non_sponsored[-10:]
    
    for B in non_sponsored:# iterates through the results Milber Insurane
        name = B.text
        profile = url[:20] + list(B.links)[0]
        # handling redirecting url 
        if "&request_id" in profile: 
            tru_url = profile[profile.index('url=https%')+len('url=https%'):profile.index('&request_id')]
            divider_mark = tru_url[tru_url.index('com')+len('com'):tru_url.index('biz')]
            profile = 'https://'+tru_url[tru_url.index('www'):].replace(divider_mark, '/')
        
        result = {"Business Name": name, "Profile": profile}
        result_list.append(result)
        
        for key, value in result.items():
            if value == '':
                result[key] = "Not Listed"
            
        result_list.append(result)
    

    results_scraped += len(result_list)
    save_index += 1
    
    m_df = pd.DataFrame(result_list)
    m_df.to_csv(f"yelp  {search_subject} in {search_location} {save_index} primary.csv", index= False)
    
    result_list = []

    try:
        file = open(f'tracker {search_subject} in {search_location} at {date_time}.txt','a') 
    except:
        file = open(f'tracker {search_subject} in {search_location} at {date_time}.txt','w')         
    file.write('\n')
    file.write (f"yelp  {search_subject} in {search_location} {save_index} primary.csv") 
    file.write('\n')
    file.write(F'{site}')   
    file.close() 
           
    try:
        previous_page = site
        next_page = response.find('div[class*="pagination-links"] > div')[-1]
        site = list(next_page.links)[0]
        next = True   
    except:
        print("scrapping is interrupted >>LAST PAGE ERROR<< !!")
        break
    sleep(0.3)
    
print(f"Primary is done, {results_scraped + len(result_list)} results Scraped successfully !!")

24 pages available.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:34<00:00,  5.80s/page]

Primary is done, 120 results Scraped successfully !!


In [9]:
file = open(f'tracker {search_subject} in {search_location} at {date_time}.txt','r')
for line in file:
    
    if f'{search_location} 1 primary' in line:
        m_df0 = pd.read_csv(str(line).strip('\n')) 
        continue 
     
    if 'https://' not in line and 'primary' in line: #and ' 0 primary' not in line
        m_df = pd.concat([m_df0, pd.read_csv(str(line).strip('\n'))], axis=0, ignore_index= True)    
        m_df0 = m_df
        
file.close()
main_df = m_df

In [10]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Business Name  120 non-null    object
 1   Profile        120 non-null    object
dtypes: object(2)
memory usage: 2.0+ KB


In [11]:
main_df.tail()

,Business Name,Profile
115,JP Accountancy,https://www.yelp.com/biz/jp-accountancy-san-fr...
116,Mauricio Midence Tax Preparation & Financial S...,https://www.yelp.com/biz/mauricio-midence-tax-...
117,Mauricio Midence Tax Preparation & Financial S...,https://www.yelp.com/biz/mauricio-midence-tax-...
118,Kugelman Law Tax & Cryptocurrency Attorneys,https://www.yelp.com/biz/kugelman-law-tax-and-...
119,Kugelman Law Tax & Cryptocurrency Attorneys,https://www.yelp.com/biz/kugelman-law-tax-and-...


In [12]:
main_df.to_csv(f"Yelp Main {search_subject} in {search_location}.csv", index= False)

In [13]:
files = os.listdir()
for file in glob.glob("*.csv"):
    if 'Main' in file or 'Secondary' in file :
        continue
    os.remove(file)

> ### Secondary Scraper Cell.

In [ ]:
# main_df = pd.read_csv('Yellow Pages Main New Car Dealers in NewYork.csv')

In [ ]:
# main_df.info()

In [14]:
""" separating the main list and the scrapped page variable to make prevent data loss when relaunching the scraper in 
    case of connection interruption
"""
contacts = []
I = 0  # power interruption index saved in the .txt file
profile_no = 0 # connection interruption index
save_index = 0
results_scraped = 0
previous_page = 'https://www.yelp.com'
breaker = False

In [15]:
start_time = time.time()

for profile in tqdm(main_df["Profile"][I:] ,unit= "profile", ncols = 160, colour= '#0A9092'):
    """ scraping the internet"""  
    ua = UserAgent()
    
    header = {
                'user agent': ua,
                'origin': 'https://www.yelp.com',
                'referer':f'{previous_page}',
                }
    while True:
        try:
            try:
                breaker = open(f'breaker.txt','r')
                breaker = True
                break
            except:
                pass
        
            session = HTMLSession()
            r = session.get(profile)
            response = r.html
            break       
        except:
            ua = UserAgent().random
            sleep(5)
            
    if breaker == True:
        break
    
    # index = main_df[main_df["Profile"]== profile] 
        
    scripts = response.xpath('//script[@type="application/json"]')[1].text
    head = scripts.index('{')
    back = len(scripts)-scripts[len(scripts)::-1].index('}')
    j_script =  scripts[head:back]                    
    data = json.loads(j_script)
        
    review_found = False
    ratting_found = False
    for D in data:
        if '.phoneNumber' in D: 
            try:
                phone = data[D]['formatted']
            except:
                pass
            
        if '.location.address' in D:    
            try:    
                address = data[D]["formatted"]
            except:
                pass    
            
        if '.externalResources.website' in D:    
            try:    
                website = 'https://www.'+ data[D]['url'].split(';')[-1]
            except:
                pass      
                        
        if ratting_found == False:    
            try:    
                ratting = int(data[D]['rating'])
                ratting_found = True
            except:
                pass 
                
        if review_found == False:    
            try:    
                review = int(data[D]['reviewCount'])
                review_found = True
            except:
                pass 
        
    B_contacts = {"Phone": phone, "Address": address, "Website": website, "Rating": ratting, "Review Count": review}
    contacts.append(B_contacts)
    
    for key, value in B_contacts.items():
        if value == '':
            B_contacts[key] = "Not Listed"
                    
    contacts.append(B_contacts)
    profile_no += 1
    
    if len(contacts) == 10: # saving the outputs to a csv file every 100 page
        results_scraped += len(contacts)
        save_index += 1
        
        s_df = pd.DataFrame(contacts)
        s_df.to_csv(f"Yelp {search_subject} in {search_location} {save_index} secondary.csv", index= False)  
        
        contacts = []
        
        file = open(f'tracker {search_subject} in {search_location} at {date_time}.txt','a')
        file.write('\n')
        file.write (f"Yelp {search_subject} in {search_location} {save_index} secondary.csv") 
        file.write('\n')
        file.write(F'{profile_no}')   
        file.close()
        
    previous_page = profile
    sleep(0.7) 
         
s_df0 = pd.DataFrame(contacts)
s_df0.to_csv(f"Yelp {search_subject} in {search_location} 0 secondary.csv", index= False)
file = open(f'tracker {search_subject} in {search_location} at {date_time}.txt','a')
file.write('\n')
file.write (f"Yelp {search_subject} in {search_location} 0 secondary.csv") 
file.write('\n')   
file.close()

print(f"Secondary is done, {results_scraped + len(result_list)} results Scraped successfully !!")

 36%|█████████████████████████████████████████▉                                                                           | 43/120 [02:59<05:21,  4.18s/profile]


IndexError: list index out of range

In [16]:
profile

'https://www.yelp.com/biz/ekco-tax-services-san-francisco?osq=accountants'

In [ ]:
file = open(f'tracker {search_subject} in {search_location} at {date_time}.txt','r')
for line in file:
    
    if f'{search_location} 1 secondary' in line:
        s_df0 = pd.read_csv(str(line).strip('\n')) 
        continue 
     
    if 'https://' not in line and 'secondary' in line: #and ' 0 secondary' not in line
        s_df = pd.concat([s_df0, pd.read_csv(str(line).strip('\n'))], axis=0, ignore_index= True)    
        s_df0 = s_df
        
file.close()
secondary_df = s_df

In [ ]:
secondary_df.info()

In [ ]:
secondary_df.tail()

In [ ]:
secondary_df.to_csv(f"Yelp Secondary {search_subject} in {search_location}.csv", index= False)

In [ ]:
files = os.listdir()
for file in glob.glob("*.csv"):
    if 'Main' in file or 'Secondary' in file :
        continue
    os.remove(file)

In [ ]:
df_combined = main_df.join(secondary_df)
df_combined.index += 1  # make the index start from one

In [ ]:
df_combined.head()

In [ ]:
df_combined.info()

In [ ]:
df_combined.to_csv(f"Yelp {search_subject} in {search_location}.csv", index= False)

In [ ]:
files = os.listdir()
for file in glob.glob("*.csv"):
    if 'Main' in file or 'Secondary' in file :
        os.remove(file)